In [115]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

In [89]:
# importing the data
tm_dummy = pd.read_csv('data/train_store_states_Python.csv')

In [90]:
tm_dummy = tm_dummy[tm_dummy['Sales'] != 0]

In [91]:
x = tm_dummy.drop(['Sales', 'Customers', 'Date', 'Unnamed: 0'], 1)
y = tm_dummy[['Store','Sales']]

In [92]:
x['PromoInterval'].fillna(0, inplace = True)
x.ix[(x['PromoInterval'] == 'Jan,Apr,Jul,Oct'), 'PromoInterval'] = 1
x.ix[(x['PromoInterval'] == 'Feb,May,Aug,Nov'), 'PromoInterval'] = 2
x.ix[(x['PromoInterval'] == 'Mar,Jun,Sept,Dec'), 'PromoInterval'] = 3

In [93]:
x_PI = pd.get_dummies(x.PromoInterval, 'PI')
x = pd.concat([x, x_PI[x_PI.columns[1:]]],axis = 1)

In [94]:
x.drop(['Promo2SinceWeek','Promo2SinceYear','PromoInterval','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'],1,inplace = True)

In [232]:
runs = 1115
y = tm_dummy[['Store','Sales']]

models = {}
print "Start!"
for i in range(1,runs):
    model = linear_model.LinearRegression()
    x_temp = x[x.Store==i].drop('Store',1)
    y_temp = np.log(y[y.Store==i].drop('Store',1))
    models['model_' + str(i)] = model.fit(x_temp, y_temp)
print "End!"

Start!
End!


In [233]:
def rmse(predictions, targets):
   return ((predictions - targets)/targets) ** 2

In [234]:
scores = {}
y = tm_dummy[['Store','Sales', 'month']]
print "Start!"
for i in range(1,runs):
    x_temp = x[(x.Store == i) & ((x.month == 8) | (x.month == 9))].drop('Store',1)
    trgts = np.array(y[(x.Store == i) & ((x.month == 8) | (x.month == 9))].drop(['Store','month'],1))
    preds = np.exp(models['model_' + str(i)].predict(x_temp))
    scores['model_' + str(i)] = rmse(preds, trgts)
print "End!"

Start!
End!


In [235]:
total = 0
for i in range(1,runs):
    total += sum(scores['model_' + str(i)])
np.sqrt(total/(runs*104))[0]

0.17850673642449896